# 전이 학습 실습 및 구현

### **data load**

주의) Google Drive 상에 바로 ants_bees.zip 파일 업로드 하고 시작하기!!

* 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


* zip 파일 현재 폴다로 복사

In [3]:
!cp /content/drive/MyDrive/ants_bees.zip ./

* 파일 압축 해제

In [4]:
!unzip ants_bees.zip

Archive:  ants_bees.zip
   creating: ants_bees/train/
   creating: ants_bees/train/ants/
  inflating: ants_bees/train/ants/0013035.jpg  
  inflating: ants_bees/train/ants/1030023514_aad5c608f9.jpg  
  inflating: ants_bees/train/ants/1095476100_3906d8afde.jpg  
  inflating: ants_bees/train/ants/1099452230_d1949d3250.jpg  
  inflating: ants_bees/train/ants/116570827_e9c126745d.jpg  
  inflating: ants_bees/train/ants/1225872729_6f0856588f.jpg  
  inflating: ants_bees/train/ants/1262877379_64fcada201.jpg  
  inflating: ants_bees/train/ants/1269756697_0bce92cdab.jpg  
  inflating: ants_bees/train/ants/1286984635_5119e80de1.jpg  
  inflating: ants_bees/train/ants/132478121_2a430adea2.jpg  
  inflating: ants_bees/train/ants/1360291657_dc248c5eea.jpg  
  inflating: ants_bees/train/ants/1368913450_e146e2fb6d.jpg  
  inflating: ants_bees/train/ants/1473187633_63ccaacea6.jpg  
  inflating: ants_bees/train/ants/148715752_302c84f5a4.jpg  
  inflating: ants_bees/train/ants/1489674356_09d48dde0a.jpg 

* 데이터 증강 및 정규화

In [5]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [6]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

* 데이터셋, 데이터로더 생성

In [7]:
import os

In [8]:
data_dir = 'ants_bees'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [9]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 244
     Root location: ants_bees/train
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=warn)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 'val': Dataset ImageFolder
     Number of datapoints: 153
     Root location: ants_bees/val
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
                CenterCrop(size=(224, 224))
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            )}

In [10]:
dataset_sizes

{'train': 244, 'val': 153}

In [11]:
class_names

['ants', 'bees']

### **model code**

In [12]:
import time
import copy

In [23]:
def train_model(model, criterion, optimizer, num_epochs=10):
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print(f"Epoch {epoch}/{num_epochs-1}")
    print("-"*10)

    # 각 epoch은 학습 단계와 검증 단계를 갖는다.
    for phase in ['train', 'val']:
      if phase == 'train':
        model.train() # 모델을 학습 모드로 설정
      else:
        model.eval()  # 모델을 평가 모드로 설정

      running_loss = 0.0
      running_corrects = 0

      # 미니 배치 학습 루프
      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # 순전파
        # 학습 시에만 연산 기록을 추적
        with torch.set_grad_enabled(phase == "train"):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          # 학습 단계인 경우 역전파 + 최적화
          if phase == "train":
            loss.backward()
            optimizer.step()

        # 통계
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects.double() / dataset_sizes[phase]

      print(f'{phase} Loss: {epoch_loss:.4f}  Acc: {epoch_acc:.4f}')

      # 모델을 깊은 복사함
      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())

    print()
  time_elapsed = time.time() - since
  print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
  print(f'Best val Acc: {best_acc:.4f}')

  # 가장 나은 모델 가중치를 불러옴
  model.load_state_dict(best_model_wts)
  return model

모델 비교

In [16]:
import torchvision.models as models
import torch.optim as optim

* ResNet18 모델 : 가중치 초기화 후 학습 진행

In [24]:
model_ft = models.resnet18(pretrained=False, num_classes=2)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
model_ft = train_model(model_ft, criterion, optimizer_ft, num_epochs=10)

Epoch 0/9
----------
train Loss: 0.8958  Acc: 0.5328
val Loss: 0.7683  Acc: 0.5621

Epoch 1/9
----------
train Loss: 0.7767  Acc: 0.5738
val Loss: 0.9543  Acc: 0.6275

Epoch 2/9
----------
train Loss: 0.8137  Acc: 0.5410
val Loss: 0.9117  Acc: 0.5294

Epoch 3/9
----------
train Loss: 0.7930  Acc: 0.5369
val Loss: 1.5438  Acc: 0.5425

Epoch 4/9
----------
train Loss: 0.7551  Acc: 0.6270
val Loss: 0.6771  Acc: 0.6797

Epoch 5/9
----------
train Loss: 0.9098  Acc: 0.5656
val Loss: 0.8639  Acc: 0.6863

Epoch 6/9
----------
train Loss: 0.7653  Acc: 0.6270
val Loss: 0.5541  Acc: 0.6732

Epoch 7/9
----------
train Loss: 0.7456  Acc: 0.5943
val Loss: 0.6436  Acc: 0.6275

Epoch 8/9
----------
train Loss: 0.8958  Acc: 0.5943
val Loss: 0.8425  Acc: 0.6797

Epoch 9/9
----------
train Loss: 0.6961  Acc: 0.6639
val Loss: 0.6929  Acc: 0.7320

Training complete in 0m 54s
Best val Acc: 0.7320


* ResNet 전이 학습

In [25]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
model_ft = train_model(model_ft, criterion, optimizer_ft, num_epochs=10)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 136MB/s]


Epoch 0/9
----------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


train Loss: 0.6618  Acc: 0.6844
val Loss: 0.2201  Acc: 0.9020

Epoch 1/9
----------
train Loss: 0.5125  Acc: 0.7664
val Loss: 0.4427  Acc: 0.8235

Epoch 2/9
----------
train Loss: 0.5028  Acc: 0.7787
val Loss: 0.2920  Acc: 0.8889

Epoch 3/9
----------
train Loss: 0.4543  Acc: 0.8115
val Loss: 0.2755  Acc: 0.9085

Epoch 4/9
----------
train Loss: 0.4398  Acc: 0.8361
val Loss: 0.3400  Acc: 0.8693

Epoch 5/9
----------
train Loss: 0.4346  Acc: 0.8402
val Loss: 0.3491  Acc: 0.8758

Epoch 6/9
----------
train Loss: 1.0579  Acc: 0.6803
val Loss: 0.4147  Acc: 0.8693

Epoch 7/9
----------
train Loss: 0.4168  Acc: 0.8197
val Loss: 0.3016  Acc: 0.8954

Epoch 8/9
----------
train Loss: 0.6594  Acc: 0.7500
val Loss: 0.6691  Acc: 0.7582

Epoch 9/9
----------
train Loss: 0.5280  Acc: 0.7992
val Loss: 0.7687  Acc: 0.8301

Training complete in 0m 51s
Best val Acc: 0.9085


* EfficientNetV2 모델 : 가중치 초기화 후 학습 진행

In [26]:
model_ft = models.efficientnet_v2_s(pretrained=False, num_classes=2)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
model_ft = train_model(model_ft, criterion, optimizer_ft, num_epochs=10)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 0/9
----------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


train Loss: 2.4772  Acc: 0.5984
val Loss: 8.8579  Acc: 0.4248

Epoch 1/9
----------
train Loss: 1.7916  Acc: 0.4959
val Loss: 23.0120  Acc: 0.5294

Epoch 2/9
----------
train Loss: 1.5412  Acc: 0.5328
val Loss: 9.8628  Acc: 0.4902

Epoch 3/9
----------
train Loss: 1.3592  Acc: 0.5164
val Loss: 3.8465  Acc: 0.4967

Epoch 4/9
----------
train Loss: 1.2324  Acc: 0.5041
val Loss: 40.4175  Acc: 0.4314

Epoch 5/9
----------
train Loss: 1.2510  Acc: 0.5246
val Loss: 9.3277  Acc: 0.5686

Epoch 6/9
----------
train Loss: 1.4379  Acc: 0.4918
val Loss: 7.8482  Acc: 0.5163

Epoch 7/9
----------
train Loss: 1.2392  Acc: 0.5615
val Loss: 1.6415  Acc: 0.4641

Epoch 8/9
----------
train Loss: 1.0718  Acc: 0.5041
val Loss: 4.3662  Acc: 0.4902

Epoch 9/9
----------
train Loss: 1.1895  Acc: 0.5492
val Loss: 1.4933  Acc: 0.5490

Training complete in 1m 48s
Best val Acc: 0.5686


* EfficientNetV2 전이 학습

In [27]:
model_ft = models.efficientnet_v2_s(pretrained=True)
num_ftrs = model_ft.classifier[1].in_features
model_ft.classifier = nn.Linear(num_ftrs, 2)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
model_ft = train_model(model_ft, criterion, optimizer_ft, num_epochs=10)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 93.1MB/s]


Epoch 0/9
----------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


train Loss: 0.5739  Acc: 0.7049
val Loss: 0.3613  Acc: 0.9412

Epoch 1/9
----------
train Loss: 0.3950  Acc: 0.8279
val Loss: 0.2106  Acc: 0.9542

Epoch 2/9
----------
train Loss: 0.3249  Acc: 0.8648
val Loss: 0.1806  Acc: 0.9412

Epoch 3/9
----------
train Loss: 0.2929  Acc: 0.8770
val Loss: 0.2232  Acc: 0.9281

Epoch 4/9
----------
train Loss: 0.2938  Acc: 0.8893
val Loss: 0.1929  Acc: 0.9542

Epoch 5/9
----------
train Loss: 0.3172  Acc: 0.8648
val Loss: 0.1430  Acc: 0.9608

Epoch 6/9
----------
train Loss: 0.3107  Acc: 0.8730
val Loss: 0.1884  Acc: 0.9477

Epoch 7/9
----------
train Loss: 0.2688  Acc: 0.8770
val Loss: 0.1657  Acc: 0.9477

Epoch 8/9
----------
train Loss: 0.3194  Acc: 0.8443
val Loss: 0.1718  Acc: 0.9412

Epoch 9/9
----------
train Loss: 0.2294  Acc: 0.8975
val Loss: 0.1843  Acc: 0.9477

Training complete in 1m 35s
Best val Acc: 0.9608
